# User Intereset Analysis

## I. Setup


In [2]:
import sys
sys.path.append('..')


In [3]:
import pandas as pd

from lib.types.dataset_type import DatasetType
from lib.types.source_type import SourceType


In [4]:
movies_df = pd.read_csv(DatasetType.movies_metadata.cleaned_path())
movies_df.rename(
    columns={'id': 'movie_id'},
    inplace=True
)
movies_df.shape


/Users/theachoem/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(45465, 18)

In [5]:
movies_df.columns


Index(['budget', 'movie_id', 'popularity', 'revenue', 'runtime', 'title',
       'vote_average', 'vote_count', 'genre_ids', 'genres', 'btc_name',
       'btc_id', 'genres1', 'genres2', 'genres3', 'genres4', 'avg_rating',
       'quantity'],
      dtype='object')

In [6]:
rating_df = pd.read_csv(DatasetType.ratings.original_path())
rating_df.rename(
    columns={'userId': 'user_id', 'movieId': 'movie_id'},
    inplace=True
)
rating_df.sample(10)


,user_id,movie_id,rating,timestamp
13454557,139925,4164,3.0,1000060850
16434787,170821,8636,4.0,1208201992
19540690,202983,1199,4.5,1259626496
10596061,109336,1961,4.0,915616916
22315434,231628,7745,4.0,1101009911
4178315,43382,318,4.0,830347958
22843956,237345,2513,3.0,959114434
9638083,99404,329,2.0,832394370
22452735,233049,610,3.0,883247038
9441300,97445,908,3.5,1476020461


## II. Find all genres


In [7]:
def find_all_genres(genres: pd.Series):
    genres = genres.str.split("|")
    genres_cleaned = [item for item in genres.to_list()
                      if isinstance(item, list)]

    all_movie_genres = "|".join([
        "|".join(i)
        for i in genres_cleaned
    ]).split("|")

    all_movie_genres = set(all_movie_genres)
    return list(all_movie_genres)


all_genres = find_all_genres(movies_df['genres'])
all_genres


['Sentai Filmworks',
 'Fantasy',
 'Family',
 'Adventure',
 'Animation',
 'GoHands',
 'War',
 'Music',
 'Carousel Productions',
 'Crime',
 'Mystery',
 'Romance',
 'Vision View Entertainment',
 'Documentary',
 'Aniplex',
 'TV Movie',
 'Foreign',
 'Telescene Film Group Productions',
 'BROSTA TV',
 'Comedy',
 'Western',
 'Mardock Scramble Production Committee',
 'Horror',
 'Drama',
 'History',
 'Science Fiction',
 'Action',
 'Thriller']

## III. Find user interest per user


In [8]:
user_id = 123963

rating_per_user_df = rating_df[rating_df['user_id'] == user_id]
rating_per_user_df['movie_id'] = rating_per_user_df['movie_id'].astype(str)

rating_per_user_df

/var/folders/g8/wgj905m158gbcnzm6q5v9y2r0000gn/T/ipykernel_13162/4066956769.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_per_user_df['movie_id'] = rating_per_user_df['movie_id'].astype(str)


,user_id,movie_id,rating,timestamp
11959259,123963,47,4.0,1485300814
11959260,123963,111,1.5,1485362108
11959261,123963,253,4.5,1485310359
11959262,123963,260,2.0,1485292966
11959263,123963,293,4.0,1485362616
...,...,...,...,...
11959584,123963,156726,5.0,1486092275
11959585,123963,160565,4.5,1485300861
11959586,123963,160571,5.0,1485293711
11959587,123963,161634,5.0,1490017471


In [9]:
movies_per_user_df = pd.merge(
    left=rating_per_user_df,
    right=movies_df,
    on='movie_id'
)

movies_per_user_df


,user_id,movie_id,rating,timestamp,budget,popularity,revenue,runtime,title,vote_average,...,genre_ids,genres,btc_name,btc_id,genres1,genres2,genres3,genres4,avg_rating,quantity
0,123963,111,1.5,1485362108,25000000,11.299673,65884703.0,170.0,Scarface,8.0,...,28|80|18|53,Action|Crime|Drama|Thriller,NaN,NaN,Action,Crime,Drama,Thriller,4.089309,111
1,123963,253,4.5,1485310359,7000000,9.773768,161777836.0,121.0,Live and Let Die,6.4,...,12|28|53,Adventure|Action|Thriller,James Bond Collection,645.0,Adventure,Action,Thriller,Adventure,3.501930,253
2,123963,260,2.0,1485292966,0,5.865697,0.0,86.0,The 39 Steps,7.4,...,28|53|9648,Action|Thriller|Mystery,NaN,NaN,Action,Thriller,Mystery,Action,4.132299,260
3,123963,293,4.0,1485362616,0,12.583826,43440294.0,123.0,A River Runs Through It,7.0,...,18,Drama,NaN,NaN,Drama,Drama,Drama,Drama,4.080149,293
4,123963,296,5.0,1485293102,200000000,20.818907,435000000.0,109.0,Terminator 3: Rise of the Machines,5.9,...,28|53|878,Action|Thriller|Science Fiction,The Terminator Collection,528.0,Action,Thriller,Science Fiction,Action,4.169975,296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,123963,88744,3.5,1486092935,0,0.511713,0.0,93.0,The Woman with the 5 Elephants,6.5,...,99|10769,Documentary|Foreign,NaN,NaN,Documentary,Foreign,Documentary,Documentary,3.556342,88744
69,123963,93840,2.0,1485308986,0,0.773371,0.0,94.0,Blind Man,5.9,...,53|28,Thriller|Action,NaN,NaN,Thriller,Action,Thriller,Thriller,3.621480,93840
70,123963,97938,5.0,1485293352,0,1.092700,0.0,87.0,Heart of the Country,6.0,...,10751|10749|18,Family|Romance|Drama,NaN,NaN,Family,Romance,Drama,Family,3.706433,97938
71,123963,98491,1.5,1485309502,0,1.090793,0.0,104.0,Wuthering Heights,7.0,...,18|10749,Drama|Romance,NaN,NaN,Drama,Romance,Drama,Drama,4.095850,98491


In [10]:
movies_per_user_df.sort_values(by=['rating'], ascending=False)


,user_id,movie_id,rating,timestamp,budget,popularity,revenue,runtime,title,vote_average,...,genre_ids,genres,btc_name,btc_id,genres1,genres2,genres3,genres4,avg_rating,quantity
13,123963,1645,5.0,1485309885,40000000,13.309013,152266007.0,149.0,A Time to Kill,7.0,...,80|18|53,Crime|Drama|Thriller,NaN,NaN,Crime,Drama,Thriller,Crime,3.516590,1645
60,123963,53129,5.0,1485364196,0,0.963791,0.0,130.0,The Turkish Gambit,6.3,...,9648|12|28|18|36,Mystery|Adventure|Action|Drama|History,The Adventures of Erast Fandorin,93446.0,Mystery,Adventure,Action,Drama,3.657163,53129
4,123963,296,5.0,1485293102,200000000,20.818907,435000000.0,109.0,Terminator 3: Rise of the Machines,5.9,...,28|53|878,Action|Thriller|Science Fiction,The Terminator Collection,528.0,Action,Thriller,Science Fiction,Action,4.169975,296
42,123963,8874,5.0,1485293830,38000000,4.678572,299288605.0,105.0,My Best Friend's Wedding,6.4,...,35|10749,Comedy|Romance,NaN,NaN,Comedy,Romance,Comedy,Comedy,3.884575,8874
7,123963,593,5.0,1485292919,0,11.059785,0.0,167.0,Solaris,7.7,...,18|878|12|9648,Drama|Science Fiction|Adventure|Mystery,NaN,NaN,Drama,Science Fiction,Adventure,Mystery,4.152246,593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,123963,8965,0.5,1486093980,0,7.751536,0.0,70.0,Atlantis: Milo's Return,5.4,...,14|16|878|10751|28,Fantasy|Animation|Science Fiction|Family|Action,Atlantis Collection,100965.0,Fantasy,Animation,Science Fiction,Family,3.107816,8965
24,123963,4368,0.5,1486094108,2500000,0.944886,0.0,94.0,Dark Wolf,3.4,...,27,Horror,NaN,NaN,Horror,Horror,Horror,Horror,2.428208,4368
19,123963,2692,0.5,1486092228,0,0.123453,0.0,90.0,The Red Elvis,8.0,...,99,Documentary,NaN,NaN,Documentary,Documentary,Documentary,Documentary,3.979886,2692
14,123963,1907,0.5,1486093221,40000000,14.430311,144056873.0,119.0,The Beach,6.3,...,18|12|10749|53,Drama|Adventure|Romance|Thriller,NaN,NaN,Drama,Adventure,Romance,Thriller,3.691785,1907


In [11]:
", ".join(movies_per_user_df.columns.to_list())

'user_id, movie_id, rating, timestamp, budget, popularity, revenue, runtime, title, vote_average, vote_count, genre_ids, genres, btc_name, btc_id, genres1, genres2, genres3, genres4, avg_rating, quantity'

In [12]:
print("HERE IS ALL GENRES THAT USER HAVE RATED:")
", ".join(find_all_genres(movies_per_user_df['genres']))


HERE IS ALL GENRES THAT USER HAVE RATED:


'Fantasy, Family, Adventure, Animation, Music, Crime, Mystery, Romance, Documentary, TV Movie, Foreign, Comedy, Western, Horror, Drama, History, Science Fiction, Action, Thriller'

## IV. Average rating for each genre

In [13]:
rating_dict = {}
avg_rating_dict = {}

for row in movies_per_user_df.values:
    movie_id = row[1]
    rating = row[2]
    genres = row[12]

    for genre in str(genres).split("|"):
        if(rating_dict.get(genre) == None):
            rating_dict[genre] = []
        rating_dict[genre].append(rating)

for key, value in rating_dict.items():
    avg_rating_dict[key] = sum(value) / len(value)

dict_sorted = sorted(
    avg_rating_dict.items(),
    key=lambda element: element[1],
    reverse=True,
)

print("HERE IS AVERAGE RATING FOR GENRE FOR USER: {}".format(user_id))
avg_rating_dict = {key: value for key, value in dict_sorted}
avg_rating_dict


HERE IS AVERAGE RATING FOR GENRE FOR USER: 123963


{'TV Movie': 4.0,
 'Adventure': 3.875,
 'Foreign': 3.6666666666666665,
 'Western': 3.5,
 'History': 3.5,
 'Romance': 3.4642857142857144,
 'Horror': 3.4444444444444446,
 'Fantasy': 3.375,
 'Science Fiction': 3.35,
 'Drama': 3.3205128205128207,
 'Action': 3.2857142857142856,
 'Comedy': 3.272727272727273,
 'Crime': 3.2666666666666666,
 'Thriller': 3.261904761904762,
 'Mystery': 3.142857142857143,
 'Documentary': 2.5,
 'Family': 2.0,
 'Music': 2.0,
 'Animation': 0.5}

## V. Find possible rating that user may rate on other genre

In [14]:
bug_dict = {}

for genres in movies_df['genres']:
    genres_list = str(genres).split("|")

    has = False
    cache = None

    for genre in genres_list:
        if(avg_rating_dict.get(genre) != None):
            has = True
            cache = genre
            break

    for genre in genres_list:
        if(bug_dict.get(genre) == None):
            bug_dict[genre] = []
        if(has):
            bug_dict[genre].append(avg_rating_dict[cache])

avg_rating_dict_v2 = {}
for key, value in bug_dict.items():
    avg_rating_dict_v2[key] = sum(
        value) / len(value) if len(value) > 0 else 0.0

avg_rating_dict_v2 = sorted(
    avg_rating_dict_v2.items(),
    key=lambda element: element[1],
    reverse=True,
)

# avg_rating_dict_v2 = {key: value for key, value in avg_rating_dict_v2}
# avg_rating_dict_v2

for key, value in avg_rating_dict_v2:
    if(avg_rating_dict.get(key) == None):
        print("{}: {}".format(key, value))


War: 3.2746854148098135
nan: 0.0
Carousel Productions: 0.0
Vision View Entertainment: 0.0
Telescene Film Group Productions: 0.0
Aniplex: 0.0
GoHands: 0.0
BROSTA TV: 0.0
Mardock Scramble Production Committee: 0.0
Sentai Filmworks: 0.0
